##JavaCall

* `Pkg.add(JavaCall)`
* Works with JDK `1.7` and `1.8`
* JVM needs to be of the same "bit-ness" as Julia
 * ie, do not mix 32-bit and 64-bit libraries
* Can call (almost) any Java methods, with any arguments

###Caveats
* Cant yet handle multi dimensional arrays
* Some segault errors (being actively hunted down)
 * But try it and see if it works for you (it does for me)
* JVM can only be loaded once per process
* Julia and Java GC's don't communicate memory pressure

In [1]:
using JavaCall

Loaded /Library/Java/JavaVirtualMachines/jdk1.7.0_76.jdk/Contents/Home/jre/lib/server/libjvm.dylib


In [2]:

tika_jar = "/Users/aviks/.julia/v0.3/Taro/deps/tika-app-1.4.jar"

JavaCall.addClassPath(tika_jar)
JavaCall.addOpts("-Xmx256M")
JavaCall.addOpts("-Djava.awt.headless=true")

JavaCall.init()

In [3]:
File = @jimport java.io.File

JavaObject{symbol("java.io.File")} (constructor with 2 methods)

In [4]:
typeof(JavaObject{symbol("java.io.File")})

DataType

https://docs.oracle.com/javase/7/docs/api/java/io/File.html

In [5]:
File = @jimport java.io.File

JavaObject{symbol("java.io.File")} (constructor with 2 methods)

In [6]:
f=File((JString,), "/Users/aviks/.julia/v0.3/Taro/test/df-test.xlsx")

JavaObject{symbol("java.io.File")}(Ptr{Void} @0x00007fe3b95c5e60)

https://poi.apache.org/apidocs/

In [7]:
WorkbookFactory = @jimport org.apache.poi.ss.usermodel.WorkbookFactory
Workbook = @jimport org.apache.poi.ss.usermodel.Workbook
Sheet = @jimport org.apache.poi.ss.usermodel.Sheet
Row = @jimport org.apache.poi.ss.usermodel.Row
Cell = @jimport org.apache.poi.ss.usermodel.Cell

JavaObject{symbol("org.apache.poi.ss.usermodel.Cell")} (constructor with 2 methods)

In [8]:
book = jcall(WorkbookFactory, "create", Workbook, (File,), f)


signal (11): Segmentation fault: 11
unknown function (ip: 330076123)


JavaObject{symbol("org.apache.poi.ss.usermodel.Workbook")}(Ptr{Void} @0x00007fe3b95c5f30)

###jcall 

* like ccall, or pycall, but object oriented
* inputs are: 
 * reciever -- class(static methods) or object (instance methods)
 * `Type` of return
 * `Tuple` of argument types
 * the arguments...
* These are the `types` of the method being called
 * ie, they should be Java Types
* Constructors are implemented using `jcall` but have some syntactic sugar
 * Don't have to specify return type

### Type correspondence
* Types are converted automatically
* Methods arguments converted from Julia to Java
 * for bitstypes, Strings, and arrays thereof
* Method return values are converted from Java to Julia
 * for primitive types and Strings
* Java composite types are stored as opaque pointers inside `JavaObject`
* Data is copied

```
typealias jint Cint
typealias jlong Clonglong
typealias jbyte Cchar
typealias jboolean Cuchar
typealias jchar Cushort
typealias jshort Cshort
typealias jfloat Cfloat
typealias jdouble Cdouble
typealias jsize jint
jprimitive = Union(jboolean, jchar, jshort, jfloat, jdouble, jint, jlong)
```


```
typealias JString JavaObject{symbol("java.lang.String")}
typealias JObject JavaObject{symbol("java.lang.Object")}
```

In [9]:
typeof(book)

JavaObject{symbol("org.apache.poi.ss.usermodel.Workbook")} (constructor with 2 methods)

In [10]:
if isnull(book) ; error("Unable to load Excel file: $filename"); end
Sheet = @jimport org.apache.poi.ss.usermodel.Sheet
sheet = jcall(book, "getSheet", Sheet, (JString,), "Sheet1") 

JavaObject{symbol("org.apache.poi.ss.usermodel.Sheet")}(Ptr{Void} @0x00007fe3b95c5f48)

In [11]:
row = jcall(sheet, "getRow", Row, (jint,), 3)

JavaObject{symbol("org.apache.poi.ss.usermodel.Row")}(Ptr{Void} @0x00007fe3b95c5f18)

In [12]:
if isnull(row); error("bad row number"); end 
cell = jcall(row, "getCell", Cell, (jint,), 1)

JavaObject{symbol("org.apache.poi.ss.usermodel.Cell")}(Ptr{Void} @0x00007fe3b95c5f08)

In [13]:
celltype = jcall(cell, "getCellType", jint, (),)

1

In [14]:
const CELL_TYPE_NUMERIC = 0;
const CELL_TYPE_STRING = 1;
const CELL_TYPE_FORMULA = 2;
const CELL_TYPE_BLANK = 3;
const CELL_TYPE_BOOLEAN = 4;
const CELL_TYPE_ERROR = 5;

In [15]:
value = jcall(cell, "getStringCellValue", JString, (),)

"b"

In [16]:
cell = jcall(row, "getCell", Cell, (jint,), 2)

JavaObject{symbol("org.apache.poi.ss.usermodel.Cell")}(Ptr{Void} @0x00007fe3b95c5ef0)

In [17]:
celltype = jcall(cell, "getCellType", jint, (),)

0

In [18]:
jcall(cell, "getNumericCellValue", jdouble, (),)

2.0

### Error Handling
* returned objects should be checked for NULL
 * `segfault` instead of `NullPointerException`
* Exceptions are propagated in most other situations

In [19]:
jlm = @jimport "java.lang.Math"
jcall(jlm, "sins", jdouble, (jdouble,), pi/2)

Exception in thread "main" java.lang.NoSuchMethodError: sins


LoadError: Error calling Java: java.lang.NoSuchMethodError: sins
while loading In[19], in expression starting on line 2